# Artificial Intelligence
## L.EIC – 3rd Year/2nd Semester
### Exercise Sheet 1
# Solving Problems by Searching

## The Two Buckets Problem

<img src="https://qph.cf2.quoracdn.net/main-qimg-45726b16b460cae0147ae8ca245a8fb0-pjlq" width="250px" height="250px" align="right">

Two buckets of capacities **c1** (e.g. 4 liters) and **c2** (e.g. 3 liters), respectively, are initially empty. The buckets do not have any intermediate markings. The only operations you can perform are:

- Fill (completely) a bucket 
- Empty a bucket.
- Pour one bucket into the other (until the target one is full or the source one is empty).

The aim is to determine which operations to carry out so that the first bucket contains exactly **n** liters (e.g. 2 litres).

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

- State representation: A state can represent a tuple (x,y) where x represents the amount of liters in bucket A and y represents the amount of liters in bucket B.
- Inital State: (0,0) since both buckets are empty
- Operators: 
    . Fill (bucket):
        -> Name: Fill
        -> Preconditions: The specified bucket must be empty.
        -> Effetcs: The specified bucket must be filled to its capacity.
        -> Cost: 1
    . Empty (bucket):
        -> Name: Empty
        -> Preconditions: The specified bucket must be full.
        -> Effetcs: The specified bucket must be emptied.
        -> Cost: 1
    . Pour (From, To):
        -> Name: Pour
        -> Preconditions: The 'From' bucket must be filled and the 'To' bucket must have enough space to receive the entire contents of the 'From' bucket.
        -> Effetcs: The contents of the 'From' bucket will be poured into the 'To' bucket until the 'To' bucket is full or the 'From' bucket is empty.
        -> Cost: 1
- Objective test: Check if the amount of liters in the first bucket is equal to the desired amount n. The state (x,y) is a goal state if x = n.

What is the size of the state space for this problem? Represent the state space by drawing the possible objective states and displaying some of the possible transitions from the initial state.

Solve the problem by hand, using tree search. What solutions have you found?

<center><div style="width: 500px"><img src="./images/3.tree_search.png" align="right"></div></center>

### Buildind a computational approach to handle the problem

To build a program to solve the buckets problem, we will implement a solution that separates the problem definition from the algorithms used to traverse the state space. This way, we can reuse our implementations of the search strategies in other problems.

#### Representing the two buckets problem as a search problem

Let's start by defining a state for the buckets problem. For that, it'll suffice to aggregate two quantities, each representing the amount of water in one of the buckets. We also define a way of printing the state.

In [4]:
class BucketState:
    c1 = 4   # capacity for bucket 1
    c2 = 3   # capacity for bucket 2
    
    def __init__(self, b1, b2):
        self.b1 = b1
        self.b2 = b2

    '''needed for the visited list'''
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash((self.b1, self.b2)) 
    ''' - '''

    def __str__(self):
        return "(" + str(self.b1) + ", " + str(self.b2) + ")"

Now we define each of the operators on states:

In [5]:
# emptying the first bucket
def empty1(state):
    if state.b1 > 0:
        return BucketState(0, state.b2)
    return None

# emptying the second bucket
def empty2(state):
    if state.b2 > 0:
        return BucketState(state.b1, 0)
    return None

# fill the first bucker
def fill1(state):
    if (state.b1 < state.c1):
        return BucketState(state.c1, state.b2)
    return None

# fill the second bucket
def fill2(state):
    if (state.b2 < state.c2):
        return BucketState(state.b1, state.c2)
    return None

def pour12_fill2(state):
    if (state.b1 >= (state.c2 - state.b2)) and (state.b1 > 0) and (state.b2 < state.c2):
        amount = state.c2 - state.b2
        return BucketState(state.b1 - amount, state.c2)
    return None

def pour12_empty1(state):
    if (state.b1 <= (state.c2 - state.b2)) and (state.b1 > 0):
        amount = state.b2 - state.b1
        return BucketState(0, state.b2 - amount)
    return None

def pour21_fill1(state):
    if ((state.c1 - state.b1) <= state.b2) and (state.b1 < state.c1):
        amount = state.c1 - state.b1
        return BucketState(state.c1, state.b2 - amount)
    return None

def pour21_empty2(state):
    if ((state.c1 - state.b1) >= state.b2) and (state.b2 > 0):
        return BucketState(state.b1 + state.b2, 0)   

# pouring from the first bucket to the second bucket
def pour1to2(state):
    if state.b1 > 0:
        amount = min(state.b1, state.c2 - state.b2)
        return BucketState(state.b1 - amount, state.b2 + amount)
    return None

# pouring from the second bucket to the first bucket
def pour2to1(state):
    if state.b2 > 0:
        amount = min(state.b2, state.c1 - state.b1)
        return BucketState(state.b1 + amount, state.b2 - amount)
    return None

The following function will aggregate all states that can be generated from a given one:

In [6]:
def child_bucket_states(state):
    new_states = []
    if(empty1(state)):
        new_states.append(empty1(state))
    if(empty2(state)):
        new_states.append(empty2(state))
    if(fill1(state)):
        new_states.append(fill1(state))
    if(fill2(state)):
        new_states.append(fill2(state))
    if(pour12_fill2(state)):
        new_states.append(pour12_fill2(state))
    if(pour12_empty1(state)):
        new_states.append(pour12_empty1(state))
    if(pour21_fill1(state)):
        new_states.append(pour21_fill1(state))
    if(pour21_empty2(state)):
        new_states.append(pour21_empty2(state))
    return new_states

Play around with the state transition operators and check if they are working properly:

In [7]:
s1 = BucketState(0, 0)
s1 = fill1(s1)
print(s1)

s2 = BucketState(0, 0)
s2 = fill2(s2)
print(s2)

# child_bucket_states(BucketState(0, 0))


(4, 0)
(0, 3)


Finally, we need to define the goal condition:

In [8]:
def goal_bucket_state(state):
    if (state.b1 == 2): 
        return True
    return False

Test your goal condition:

In [9]:
s1 = goal_bucket_state(BucketState(2, 0))
print(s1)

s2 = goal_bucket_state(BucketState(0, 0))
print(s2)

True
False


#### Implementing search algorithms

Let us start by defining an appropriate structure to represent a node in a search tree. Each tree node will include:
- a state of the problem
- a link to its parent (to allow traveling from a leaf node towards the root of the tree)
- a list of child nodes

In [10]:
# A generic definition of a tree node holding a state of the problem
class TreeNode:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        if self.parent is None:
            self.depth = 0
        else:
            self.depth = self.parent.depth + 1

    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parent = self

##### Breadth-first search

Based on this structure, we can now implement breadth-first search. Note that we want the implementation to be independent of the problem at hand (in this case, the two buckets problem).

In [11]:
from collections import deque

def breadth_first_search(initial_state, goal_state_func, operators_func):
    root = TreeNode(initial_state)   # create the root node in the search tree
    queue = deque([root])   # initialize the queue to store the nodes
    
    while queue:
        node = queue.popleft()   # get first element in the queue
        if goal_state_func(node.state):   # check goal state
            return node
        
        for state in operators_func(node.state):   # go through next states
            # create tree node with the new state
            child = TreeNode(state, node)
            
            # link child node to its parent in the tree
            node.add_child(child) 
            
            # enqueue the child node
            queue.append(child)
            

    return None

We can now use this function to actually perform a breadth-first search on the buckets problem: we pass it the initial state, our goal condition function, and the function for obtaining child states.

In [12]:
goal = breadth_first_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states)
print(goal.state)

(2, 3)


In order to print the actual steps from the initial state to the last, we can take advantage of each node's link to its parent.

In [13]:
def print_solution(node):
    if (node == None): return
    print_solution(node.parent)
    print(node.state)
    

Now we can print the solution:

In [14]:
print_solution(goal)

(0, 0)
(4, 0)
(1, 3)
(1, 0)
(0, 1)
(4, 1)
(2, 3)


If we need a description for each of the employed operators, we could have each operation function return also such a description, and modify the TreeNode class so that each node also includes a description of the edge to get there. We leave that as an exercise after class.

##### Depth-first search

Implement depth-first search (again, in a manner that is independent of the problem at hand). You can start from your breadth-first search implementation and with minor changes get an implementation for depth-first search.

In [15]:
visited = []
def depth_first_search(initial_state, goal_state_func, operators_func): 
    node = TreeNode(initial_state)
    if goal_state_func(node.state): # check goal state
        return node

    if node.state not in visited:
        visited.append(node.state)

        for state in operators_func(node.state):   # go through next states
            new_node = TreeNode(state) # create tree node with the new state
            new_node.parent = node # associate the new node with the parent node (node is the parent of new_node)

            result = depth_first_search(new_node.state, goal_state_func, operators_func) # executes a DFS on the new_node
            if result is not None:
                return result


    return None

Test it on the two buckets problem.

In [16]:
goal = depth_first_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states)
print(goal.state)

(2, 0)


If you are unable to get a solution, think about it: depth-first search is not a complete search method, and one of the reasons for that is if the state space contains cycles. As such, you need to make sure you avoid entering into a cycle by keeping a visited nodes list or set and checking that list whenever you generate a new state.

##### Depth-limited search

Another way to make it work is to impose a depth limit to the problem. Implement depth-limited search.

In [110]:
def depth_limited_search(initial_state, goal_state_func, operators_func, depth_limit, visited):
    node = TreeNode(initial_state)
    
    if goal_state_func(node.state): # check goal state
        return node
    
    if (depth_limit == 0): return None
    
    if node.state not in visited:
        visited.append(node.state)

        for state in operators_func(node.state):   # go through next states
            new_node = TreeNode(state) # create tree node with the new state
            new_node.parent = node # associate the new node with the parent node (node is the parent of new_node)

            result = depth_limited_search(new_node.state, goal_state_func, operators_func, depth_limit - 1, visited) # executes a DFS on the new_node
            if result is not None:
                return result


    return None

Test it on the two buckets problem.

In [111]:
goal = depth_limited_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states, 
                            6, [])

if (goal != None): 
    print(goal.state)
else:
    print(goal)

(2, 3)


##### Iterative deepening search

Based on depth-limited, you can easily implement iterative-deepening search.

In [108]:
def iterative_deepening_search(initial_state, goal_state_func, operators_func):
    for i in range(1, 1000):
        node = TreeNode(initial_state)
        result = depth_limited_search(node.state, goal_state_func, operators_func, i, [])
        if result is not None:
            return result
        
    return None

Again, test it on the two buckets problem.

In [113]:
goal = iterative_deepening_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states)

if (goal != None): 
    print(goal.state)
else:
    print(goal)


(2, 3)


##### Greedy search

Start by defining an heuristic function and implement the Greedy Search algorithm (independent of the problem at hand)

In [117]:
def heuristic_bucket(node):
    # heuristic function for the bucket filling problem
    # your code here
    return abs(node.b1 - 2)


In [126]:
def get_min_bucket(array):
    min = 9999999
    bucket = array[0]
    for t in array:
        if (t[0] <= min):
            min = t[1].b1
            bucket = t
    
    return bucket


def greedy_search(initial_state, goal_state_func, operators_func, heuristic_func):
    opened_list = [(heuristic_bucket(initial_state), initial_state)]
    closed_list = []

    while opened_list:
        bucket = get_min_bucket(opened_list) # state = min(opened_list, key = lambda t: t[0])[1]
        state = bucket[1]

        if (goal_state_func(state)): return state
        
        opened_list.remove(bucket)
        closed_list.append(state) # add the current state to the closed list of states already analyzed
        
        for op in operators_func(state): # possible final states after operations in the current state
            if op in closed_list:
                continue # ignore operation
            h = heuristic_func(op)
            opened_list.append((h, op))

    return None

Test it on the two buckets problem.

In [119]:
# your code here
solution = greedy_search(BucketState(0, 0), lambda s: goal_bucket_state(s), child_bucket_states, lambda s: heuristic_bucket(s))
print(solution)

(2, 0)


##### A* Algorithm

Reuse the heuristic function defined before. This is very similar to greedy search, the difference is that it takes into account the cost of the path so far

In [127]:
def a_star_search(initial_state, goal_state_func, operators_func, heuristic):
    # your code here
    return greedy_search(initial_state, goal_state_func, operators_func, lambda s: 1 + heuristic(s))

Again, test it on the two buckets problem.

In [129]:
# your code here
solution = a_star_search(BucketState(0, 0), lambda s: goal_bucket_state(s), child_bucket_states, lambda s: heuristic_bucket(s))

if (goal != None): 
    print(solution)
else:
    print(None)

(2, 0)


## The Missionaries and Cannibals Problem 

<img src="https://www.gamezkingdom.com/content/images/thumbs/0002926_missionaries-and-cannibals.jpeg" width="250px" height="250px" align="right">

Three missionaries and three cannibals are on one of the banks of the river with a boat that only takes one or two people. The boat cannot travel the river alone.

The goal is to find a way to get the six to the other bank of the river without ever leaving more cannibals than missionaries on one of the banks (even at the instant they leave/join the boat) during the process.

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

Solve the problem by hand, using tree search. What solutions have you found?

Represent the problem as a search problem and take advantage of the implemented search algorithms to find solutions!

In [ ]:
# your code here


## N-Puzzle Problem

The objective of this exercise is the application of search methods, with emphasis on informed
search methods and the A\* algorithm, to solve the well-known N-Puzzle problem. The desired
objective self for the puzzle is as follows (0 represents the empty space):

<table>
<tr><th>9Puzzle</th><th>16Puzzle</th></tr>
<tr>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 2   | 3   |
| 4   | 5   | 6   |
| 7   | 8   | 0   |


</td>
<td>

|     |     |     |     |
| --- | --- | --- | --- |
| 1   | 2   | 3   | 4   |
| 5   | 6   | 7   | 8   |
| 9   | 10  | 11  | 12  |
| 13  | 14  | 15  | 0   |

</td>
</tr>
</table>

Starting from a given initial state, the goal is to determine which operations to perform to
solve the puzzle, reaching the desired objective self.

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

Represent the problem as a search problem and take advantage of the implemented search algorithms to find solutions!

For the Greedy Search and the A* Algorithm suppose the following heuristics for these methods:
- H1 - Number of incorrect placed pieces;
- H2 - Sum of manhattan distances from incorrect placed pieces to their correct places. 

Finally Compare the results obtained concerning execution time and memory space occupied in solving the following problems using the previous methods

<table>
<tr><th>Prob. 1</th><th>Prob. 2</th><th>Prob. 3</th><th>Prob. 4</th></tr>
<tr>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 2   | 3   |
| 5   | 0   | 6   |
| 4   | 7   | 8   | 

</td>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 3   | 6   |
| 5   | 2   | 0   |
| 4   | 7   | 8   | 

</td>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 6   | 2   |
| 5   | 7   | 3   |
| 0   | 4   | 8   | 

</td>
<td>

|     |     |     |     |
| --- | --- | --- | --- |
| 5   | 1   | 3   | 4   |
| 2   | 0   | 7   | 8   |
| 10  | 6   | 11  | 12  |
| 9   | 13  | 14  | 15  |

</td>
</tr>
</table>

In [ ]:
# the following arrays represent the problem for testing
initial_states = [
    [[1, 2, 3], [5, 0, 6], [4, 7, 8]],
    [[1, 3, 6], [5, 2, 0], [4, 7, 8]],
    [[1, 6, 2], [5, 7, 3], [0, 4, 8]],
    [[5, 1, 3, 4], [2, 0, 7, 8], [10, 6, 11, 12], [9, 13, 14, 15]]
]

In [ ]:
# your code here
